# 모델 추론 로컬 테스트

- 이 노트북은 Hugging Face TGI (Text Generation Interface) 의 도커 이미지를 다운로드 받아서,이전의 파인 튜닝된 모델을 도커 컨테이너로 테스트를 합니다.

### 필수 사항
- 파인 튜닝된 모델이 있어야 하기에, 아래의 노트북을 먼저 실행을 하셔야 합니다.
    - [01-naver-news-fsdp-QLoRA/03-SageMaker-Training.ipynb](../01-naver-news-fsdp-QLoRA/03-SageMaker-Training.ipynb) 


## 1. 환경 구성
01-naver-news-fsdp-QLoRA/03-SageMaker-Training.ipynb 노트북에서 model_s3_path 에 저장한 경로를 가져 옵니다.

In [1]:
%store -r model_s3_path
print("model_s3_path: ", model_s3_path)


model_s3_path:  {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/output/model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


## 2. 모델 아티펙트 다운로드 
이전에 훈련한 파인 튜닝 모델의 모델 아티펙트를 다운로드 합니다.
- 모델 다운로드는 아래 경로 입니다. 사용하시는 환경이 다르면 수정 하세요.
    - ```model_data_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download" ```


In [4]:
import os

model_data_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download"
os.makedirs(model_data_dir, exist_ok=True)
print("model_data_dir: ", model_data_dir)

model_data_dir:  /home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download


In [5]:
artifact_path = model_s3_path["S3DataSource"]["S3Uri"]
print("artifact_path: ", artifact_path)

artifact_path:  s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/output/model/


In [6]:
%%sh -s {artifact_path} {model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir --recursive

ls -al $model_data_dir



s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/output/model/
/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/output/model/config.json to ../../../../../../../models/llama-3-8b-naver-news-s3-download/config.json
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/output/model/generation_config.json to ../../../../../../../models/llama-3-8b-naver-news-s3-download/generation_config.json
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/output/model/model-00002-of-00007.safetensors to ../../../../../../../models/llama-3-8b-naver-news-s3-download/model-00002-of-00007.safetensors
download: s3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-27-07-50-0-2024-07-27-07-50-10-938/

## 3. HF TGI 도커 컨테이너 실행 예시 보기

#### HF TGI Docker Container Shell 보기

In [7]:
! cat /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook/02-naver-news-llama3-inference/local_test/run_tgi_docker_container.sh


aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

# Get TGI image
docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04 

# export env. vars 
export MODEL_REPO_DIR=/home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download
export MODEL_LOG_DIR=/home/ec2-user/SageMaker/tmp/log

mkdir -p $MODEL_LOG_DIR

docker run -it --runtime=nvidia --gpus all --shm-size 12g \
 -v $MODEL_REPO_DIR:/opt/ml/model:ro \
 -v $MODEL_LOG_DIR:/opt/djl/logs \
 -p 8080:8080 \
 -e HF_MODEL_ID=/opt/ml/model \
 -e SM_NUM_GPUS=4 \
 -e MESSAGES_API_ENABLED=true \
 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04 

# if needed, comment out and run as a test
# curl http://localhost:8080/generate \
#     -X POST \
#     -d '{"inputs":"What is the capital of France?","paramete

### docker container shell 실행
아래는 run_docker_container_shell 를 실행한 화면 입니다.
- 첫 번째는 docker image 를 ECR 로 부터 다운로드 합니다.
    - ![run_docker_container_shell.png](img/run_docker_container_shell.png)
- 두 번째는 tgi docker container 실행 하고, 요청을 받기 위해 준비된 상태 입니다.
    - ![run_tgi_container.png](img/run_tgi_container.png)
    - ![loading_tgi.png](img/loading_tgi.png)
    - ![ tgi_container_ready.png](img/tgi_container_ready.png)
- 이렇게 실행된 docker container 를 root 로 접근해서 컨테이너 내부를 보겠습니다.
    - ![TGI_container_inside.png](img/TGI_container_inside.png)

## 4. HF TGI 도커 컨테이너 실행 직접 하기

단계 1: Terminal 를 오픈 합니다. <br>
--> 왼쪽 상단의 File >> New >> Terminal 차례대로 선택 합니다. <br><br>
단계 2: 아래 처럼 해당 경로에 S3 에서 다운로드한 모델 파일들이 있는지 확인 합니다.
```
ls /home/ec2-user/SageMaker/models/llama-3-8b-naver-news-s3-download
```
단계 3: 아래 처럼 해당 경로로 이동 합니다.
```
cd /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook/02-naver-news-llama3-inference/local_test
```
단계 4: 아래 처럼 shell 을 실행 합니다.
```
./run_tgi_docker_container.sh 
```
위의 실행 화면과 같이 나오면 정상 입니다.

## 5. 도커 실행 테스트

In [8]:
import requests
import json

def docker_inference(endpoint,headers, payload):
    response = requests.post(
        endpoint, headers=headers, json=payload)
    return response


endpoint = "http://127.0.0.1:8080/generate"
headers = {"content-type": "application/json"}


### Completion 스타일 프롬프트 실행 테스트

In [9]:
payload = {"inputs":"What is the capital of France?","parameters":{"max_new_tokens":50}}
response = docker_inference(endpoint,headers, payload)
result = json.loads(response.text)
result
#json.dumps(result, indent=2)

{'generated_text': ' Paris, of course! But what about the capital of the world? That’s a little more difficult to answer. There are many contenders for the title, but there is no clear consensus. Some people might say that New York City is the capital of'}

### Message Style 프롬프트 실행 테스트
- 아래는 현재 에러 발생. 

In [10]:
payload = {'messages': [{'role': 'system',
   'content': 'You are a helpful assistant and write only in English'},
  {'role': 'user', 'content': 'How to make cake?'}],
 'model': 'meta-llama-3-fine-tuned',
 'parameters': {'max_tokens': 512,
  'top_p': 0.9,
  'temperature': 0.6,
  'stop': ['<|eot_id|>']}}
response = docker_inference(endpoint,headers, payload)
response
# result = json.loads(response.text)
# result

<Response [422]>

In [11]:
response.text

'Failed to deserialize the JSON body into the target type: missing field `inputs` at line 1 column 281'

## 6. 도커 실행 중지
아래와 같이 명령어를 제공하여 컨테이너를 중지 합니다.
- ![docker_stop.png](img/docker_stop.png)

축하 합니다. 로컬에서 TGI Docker Container 테스트가 완료 되었습니다.  ^^